In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
from time import sleep
from lxml import html  
import requests
import json
import argparse
from collections import OrderedDict
import lxml.html as lh
import copy
import pandas as pd
import urllib3
!pip install --upgrade -q pygsheets
!pip install --upgrade -q gspread
from google.colab import drive
drive.mount('drive')
import gspread
import pygsheets

     |████████████████████████████████| 102kB 3.7MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [2]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('TICS').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
data = pd.DataFrame.from_records(rows, index = None)
cmpList = data[data.columns[0]].astype(str).values.tolist()
print('Number of tickets', len(cmpList))

Number of tickets 100


In [0]:
sleep_val = 1

In [0]:
# for parsing yahoo finance data
#items = ['/financials?p=', '/balance-sheet?p=', '/cash-flow?p=']

financialslen = 5
balancesheetlen = 4
cashflowlen = 5

def getFinanceData(url, name):
  global financialslen
  global balancesheetlen
  global cashflowlen
  if name  == "financials":
    dates_len = financialslen
  elif name  == "balance-sheet":
    dates_len = balancesheetlen
  elif name  == "cash-flow":
    dates_len = cashflowlen
  try:
    #urllib.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    response = requests.get(url, verify=False)
    sleep(sleep_val)
    parser = html.fromstring(response.text)
    finance_table = parser.xpath('//div[contains(@class,"Pos(r)")]')
    keys = []
    values = []
    
    columnLen = 0 
    for table_data in finance_table:
      raw_table_key = table_data.xpath('.//span//text()')
      if len(raw_table_key) > 1 and raw_table_key[0] == "Breakdown":
        dates_df = pd.DataFrame(raw_table_key[1: dates_len + 1])
        return dates_df.T
    return 
  except:
    return pd.DataFrame()
      

In [0]:
def writeDatesYahooFinancial(cmpList, mode, old_len_df, writer): 
  # Parsing data from yahoo (financials)
  print("\nWriting Yahoo Financials")
  towriteCmpList = []
  ignored_cmp = []
  finalcolsize = 14

  items = ['/financials?p=', '/balance-sheet?p=', '/cash-flow?p=']
  urlPre = 'https://finance.yahoo.com/quote/'

  columns = None


  final_df = pd.DataFrame()

  for cmp in cmpList:
    cmp_df_list = []
    print('\nCompany: ', cmp)
    
    for item in items:
      url = urlPre + cmp + item + cmp
      # dataframe_list = parse_url(url, cmp)
    
      name = item[1:len(item) - 3]
      count = 1
      item_df = getFinanceData(url, name)
      cmp_df_list.append(item_df)
      
  #     filename = cmp + '_' + name + '.csv'
  #     item_df.to_csv(filename, index = False)
  #     !cp $filename drive/My\ Drive/Research/AB/
    
    
  
    cmp_df = pd.DataFrame()
    for df in cmp_df_list:
      cmp_df = pd.concat([cmp_df,df], axis = 1)
    
    # Causes problem when company name is "F" and it gets replaced everywhere in the column names. Better to do in parse_url
    # cmp_df.columns = cmp_df.columns.str.replace(cmp, "cmp")
    
    if columns == None:
      columns = cmp_df.columns.tolist()
  
    #cmp_df.sort_index(axis=1, inplace=True) # not required if string replace done at parsing time
    
    
    if cmp_df.shape[1] != finalcolsize:
      # if first cmp, size of columns should match else exact columns match should return always True. Else column mistmatch (ignore this company)
      print('Column mismatch for company: ', cmp, ' Ignoring...')
      #print('differences: ', cmp_df.columns.difference(final_df.columns), final_df.columns.difference(cmp_df.columns)) #should always print 0
      ignored_cmp.append(cmp)
      continue
    
    towriteCmpList.append(cmp)
    
    final_df = pd.concat([final_df, cmp_df], ignore_index = True)


  final_df.insert(loc=0, column='company', value=towriteCmpList)
  print('New Data: ',final_df.shape)

  filename = 'Dates_YahooFinances.xlsx'
  if mode == "new":
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    final_df.to_excel(writer, index = False)
    len_df  = len(final_df)
  elif mode == "add":
    final_df.to_excel(writer, startrow=old_len_df+1, header=None, index = False)
    len_df = old_len_df + len(final_df)

  writer.save()
  !cp $filename drive/My\ Drive/AB/

  if len(ignored_cmp) > 0:
    ignored_csv = pd.DataFrame(ignored_cmp)
    ignored_filename = 'Dates_YahooFinances_ignoredcmp.xlsx'
    ignored_csv.to_excel(ignored_filename, index="False")
    !cp $ignored_filename drive/My\ Drive/AB/
    print('IGNORED COMPANIES: ', len(ignored_cmp))
    raw = input("Enter 't' to retry or 'f' to stop execution")
    if raw == "t":
      writeDatesYahooFinancial(ignored_cmp, "add", len_df, writer)
    else:
      return
  else:
    print('IGNORED COMPANIES: ', len(ignored_cmp))

In [6]:
writeDatesYahooFinancial(cmpList, "new", 0, None)


Writing Yahoo Financials

Company:  A


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Company:  AA


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)



Company:  AAL


...

New Data:  (98, 15)
IGNORED COMPANIES:  2
Enter 't' to retry or 'f' to stop executiont

Writing Yahoo Financials

Company:  AST


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Column mismatch for company:  AST  Ignoring...

Company:  BABY


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Column mismatch for company:  BABY  Ignoring...
New Data:  (0, 1)
IGNORED COMPANIES:  2
Enter 't' to retry or 'f' to stop executionf
